In [1]:
import pandas as pd
import numpy as np

In [7]:
# 读取分类表
df_category = pd.read_excel('资产配置分类表.xlsx',dtype={'基金代码': np.object, '分类ID': np.int64})
# df_category.基金代码.unique()
# 基金名称	基金简称	基金代码	一级分类	二级分类	三级分类	分类ID	市场

In [58]:
user_name = '康力泉'
df_divid = pd.read_excel('李淑云_divid.xlsx', index_col=0)
df = pd.read_excel('李淑云_trade.xlsx', index_col=0)

record_keys = ['id', 'date', 'time', 'code', 'name', 'deal_type', 
               'nav_unit', 'nav_acc', 'volume', 'deal_money', 'fee', 
               'occur_money', 'account', 'category1', 'category2', 
               'category3', 'category_id', 'unique_id', 'note']

# 分红信息
df_divid = df_divid[~df_divid['fundName'].str.contains('货币')]
df_divid = df_divid.replace(['--'],[0])
df_divid = df_divid.reset_index(drop=True)
df_divid['id'] = df_divid.index + 1
df_divid['date'] = df_divid['confirmDate']
df_divid['time'] = '14:59:00'
df_divid['code'] = df_divid['fundCode']
df_divid['code'] = df_divid['code'].apply(lambda x: str(x).zfill(6))
df_divid['name'] = df_divid['fundName']
df_divid['deal_type'] =  '分红'
df_divid['nav_unit'] = df_divid['nav']
df_divid['nav_acc'] = df_divid['nav']
df_divid['volume'] = df_divid['dividendVol']
df_divid['deal_money'] = df_divid['dividendAmount']
df_divid['fee'] = 0.0
df_divid['occur_money'] = df_divid['dividendAmount']
df_divid['account'] = '{0}_天天'.format(user_name)
df_divid['unique_id'] = df_divid['name'] + '_' + df_divid['date'] + '_' + df_divid['dividendMethod']
df_divid['note'] = '无'
# 补充三级分类
df_divid = pd.merge(df_divid, df_category, left_on='code', right_on='基金代码', how='left')
df_divid = df_divid.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
df_divid = df_divid[record_keys]
# df_divid

def deal_type_calc(x):
    op = x['businType']
    if op in ['买基金']:
        return '买入'
    elif op in ['卖基金','卖基金极速回活期宝','超级转换-转出','卖基金回活期宝']:
        return '卖出'
    elif op in ['强增']:
        return '分红'
    else:
        return op

def deal_money_calc(x):
    money = x['cfmAmount']
    fee = x['fee']
    op = x['deal_type']
    if op == '买入':
        return round(money - fee, 2)
    elif op == '卖出':
        return round(money, 2)
    else:
        return money

def occur_money_calc(x):
    money = x['cfmAmount']
    fee = x['fee']
    op = x['deal_type']
    if op == '买入':
        return round(money, 2)
    elif op == '卖出':
        return round(money - fee, 2)
    else:
        return money
    
# 交易记录
df = df[~df['fundName'].str.contains('货币')]
# 非货币基金的操作有，businType：
# ['卖基金', '买基金', '修改分红方式', '卖基金极速回活期宝', '强增', '跨分账户份额转卡', '超级转换-转出', '转托管']
ignore_types = ['修改分红方式', '跨分账户份额转卡', '转托管']
# 去掉一些和成本无关的操作（转托管记录有 addition.json 代劳）
df = df[~df['businType'].isin(ignore_types)]
df = df.reset_index(drop=True)
df['id'] = df.index + 1
df['date'] = df['transactionCfmDate']
df['time'] = '14:59:00'
df['code'] = df['fundCode']
df['code'] = df['code'].apply(lambda x: str(x).zfill(6))
df['name'] = df['fundName']
df['deal_type'] =  df.apply(deal_type_calc, axis=1)
df['nav_unit'] = df['nav']
df['nav_acc'] = df['nav']
df['volume'] = df['cfmVol']
df['fee'] = df['charge']
df['deal_money'] = df.apply(deal_money_calc, axis=1)
df['occur_money'] = df.apply(occur_money_calc, axis=1)
df['account'] = '{0}_天天'.format(user_name)
df['unique_id'] = df['serialNo']
df['note'] = '无'
# 补充三级分类
df = pd.merge(df, df_category, left_on='code', right_on='基金代码', how='left')
df = df.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
df = df[record_keys]
# df
# print(df.businType.unique())
# df[df.businType == '卖基金回活期宝']

# 合并
df_results = pd.concat([df, df_divid], ignore_index=True)
df_results = df_results.sort_values(['date','code','category_id', 'code'])
df_results = df_results.reset_index(drop=True)
df_results['id'] = df_results.index + 1
df_results


,id,date,time,code,name,deal_type,nav_unit,nav_acc,volume,deal_money,fee,occur_money,account,category1,category2,category3,category_id,unique_id,note
0,1,2018-05-28,14:59:00,001064,广发中证环保ETF联接A,买入,0.6865,0.6865,581.97,399.52,0.48,400,康力泉_天天,A 股,行业股,中证环保,143,20180528273030013061407,无
1,2,2018-05-28,14:59:00,001064,广发中证环保ETF联接A,买入,0.6865,0.6865,581.97,399.52,0.48,400,康力泉_天天,A 股,行业股,中证环保,143,20180528273030013061414,无
2,3,2018-05-28,14:59:00,004752,广发中证传媒ETF联接A,买入,0.9191,0.9191,434.69,399.52,0.48,400,康力泉_天天,A 股,行业股,中证传媒,144,20180528273030013061409,无
3,4,2018-05-28,14:59:00,162413,华宝中证1000指数分级,买入,1.002,1.002,398.72,399.52,0.48,400,康力泉_天天,A 股,中小盘股,中证1000,123,201805289898180528303000010039,无
4,5,2018-05-28,14:59:00,162711,广发中证500ETF联接A,买入,1.2605,1.2605,316.95,399.52,0.48,400,康力泉_天天,A 股,中小盘股,中证500,121,201805289898180528303000049995,无
5,6,2018-05-31,14:59:00,162711,广发中证500ETF联接A,买入,1.2139,1.2139,1645.61,1997.6,2.40,2000,康力泉_天天,A 股,中小盘股,中证500,121,201805319898180531303000031391,无
6,7,2018-06-20,14:59:00,162711,广发中证500ETF联接A,买入,1.1108,1.1108,1079.01,1198.56,1.44,1200,康力泉_天天,A 股,中小盘股,中证500,121,201806209898180620303000181636,无
7,8,2018-06-20,14:59:00,340001,兴全可转债混合,买入,1.004,1.004,398,399.6,0.40,400,康力泉_天天,债券,国内债券,可转债,512,20180620343030004211260,无
8,9,2018-06-20,14:59:00,502010,易方达证券公司分级,买入,0.8457,0.8457,472.51,399.6,0.40,400,康力泉_天天,A 股,行业股,证券公司,145,201806209999180620303000053115,无
9,10,2018-07-06,14:59:00,001051,华夏上证50ETF联接A,买入,0.812,0.812,492.02,399.52,0.48,400,康力泉_天天,A 股,大盘股,上证50,111,20180706033032018070618451157,无
